# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [22]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.utilities import get_primary_metrics
from azureml.widgets import RunDetails

import joblib
import os
import pandas as pd

In [5]:
# Set pandas options to be able to display all columns
pd.set_option("display.max_columns", 35)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I am using the IBM HR Analytics Employee Attrition & Performance dataset from Kaggle (https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset) for the binary classification task of predicting whether an employee is at risk to leave the company, a phenomenon that is called attrition. The dataset contains the target class "Attrition" as well as demographic, job-specific, geographic and socioeconomic feature columns. It is a fictional dataset created by IBM Data Scientists.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ws = Workspace.from_config()

# Create an experiment
experiment_name = 'attrition-automl'
experiment=Experiment(ws, experiment_name)

# Retrieve the data from the workspace
dataset = Dataset.get_by_name(ws, name='attrition')
df = dataset.to_pandas_dataframe()

# Show the first rows
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,True,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,True,True,11,3,1,80,0,8,0,1,6,4,0,5
1,49,False,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,True,False,23,4,4,80,1,10,3,3,10,7,1,7
2,37,True,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,True,True,15,3,2,80,0,7,3,3,0,0,0,0
3,33,False,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,True,True,11,3,3,80,0,8,3,3,8,7,3,0
4,27,False,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,True,False,12,3,4,80,1,6,3,3,2,2,2,2


## Compute Target

In [8]:
compute_cluster_name = "cpu-cluster"

# Retrieve the compute target
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, using it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, using it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.
- "enable_early_stopping" is set to true to discard non-promising training runs.
- "enable_onnx_compatible_models" is set to true as onnx is a great format for standardized model deployment and I want to use it as part of the project
- "experiment_timeout_minutes" is set 60 to give the automl experiment 60min of training time.
- "max_concurrent_iterations" is set to 4 as this number should be smaller than the node count of the cluster (which is 5) but should be leveraged to a maximum for parallel model training.
- AUC_weighted is used as "primary_metric" because it can account for imbalanced classes (which we face here) in contrast to simple accuracy.
- As the dataset has between 1,000 and 20,000 rows, 3-fold cross validation will be used for the automated model evaluation by default (since no validation_data or n_cross_validations is specified).
- "featurization" is set to auto for automatic featurization.

In [10]:
# Check available primary metrics for classification
get_primary_metrics("classification")

['norm_macro_recall',
 'average_precision_score_weighted',
 'precision_score_weighted',
 'AUC_weighted',
 'accuracy']

In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "enable_early_stopping": True,
    "enable_onnx_compatible_models": True,
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 4,
    "primary_metric" : "AUC_weighted"
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = compute_target,
    task = "classification",
    training_data = dataset,
    label_column_name = "Attrition",
    path = "./automl",
    featurization = "auto",
    **automl_settings
)

In [14]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
automl_run.wait_for_completion()

{'runId': 'AutoML_169df40b-a192-427a-9e6b-74822c261a15',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-16T19:46:56.308959Z',
 'endTimeUtc': '2020-12-16T20:14:06.991977Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"attrition-automl","subscription_id":"bf088f59-f015-4332-bd36-54b988be7c90","resource_group":"sbirk-aml-rg","workspace_name":"sbirk-aml-ws","region":"westeurope","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
best_automl_run, automl_model = automl_run.get_output()
print(best_automl_run)
print(automl_model)

Run(Experiment: attrition-automl,
Id: AutoML_169df40b-a192-427a-9e6b-74822c261a15_49,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                penalty='l2',
                                                                                                random_state=None,
                                     

In [24]:
#TODO: Save the best model
os.makedirs("./models", exist_ok=True)
joblib.dump(automl_model, "models/automl_model.pkl")

['models/automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service